In [ ]:
# import libraries
import pandas as pd
import random
from datetime import datetime, timedelta

: 

In [ ]:
# Define Correct OWASP WSTG Categories
owasp_wstg_categories = [
    "Information Gathering",
    "Configuration and Deployment Management",
    "Identity Management",
    "Authentication Testing",
    "Authorization Testing",
    "Session Management Testing",
    "Data Validation Testing",
    "Error Handling",
    "Cryptography Testing",
    "Business Logic Testing",
    "Client-Side Testing",
    "API Testing"
]


: 

In [ ]:
# Generate synthetic test cases with OWASP WSTG details
def generate_test_cases(num_cases=100):
    test_cases = []
    for i in range(1, num_cases + 1):
        test_id = f'TEST-{i:03}' #Generates a test case ID in the format "TEST-001", "TEST-002", ..., "TEST-100".
        project_id = f'PROJ-{random.randint(1, 10):03}' 
        tester_id = f'USER-{random.randint(1, 5):03}'
        days_allocated = random.randint(1, 30) 
        owasp_category = random.choice(owasp_wstg_categories) 
        
        test_case = { #as per preferred schema provided 
            'Test ID': test_id,
            'Project ID': project_id,
            'Tester ID': tester_id,
            'Days Allocated': days_allocated,
            'OWASP Category': owasp_category,
            'Status': random.choice(['Pending', 'In Progress', 'Completed']),
            'Created On': (datetime.now() - timedelta(days=random.randint(1, 30))).strftime('%Y-%m-%d'),
            'Modified On': datetime.now().strftime('%Y-%m-%d')
        }
        test_cases.append(test_case)
    return pd.DataFrame(test_cases)

In [ ]:
# Generate updated test case data
test_case_data = generate_test_cases(100)

# Display first few rows
test_case_data.head()

In [ ]:
# Save the generated test data to a CSV file
test_case_data.to_csv("security_test_cases.csv", index=False)
print("Security test cases saved as 'security_test_cases.csv'")